In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('final_frame_pl_v4.csv', dtype={'Season': str})
data = data.dropna(subset=['FTR'])
data

Date     HomeTeam  ... Bet_winner  Index
0      1995-08-19 00:00:00  Aston Villa  ...          0   9500
1      1995-08-19 00:00:00    Blackburn  ...          0   9501
2      1995-08-19 00:00:00      Chelsea  ...          0   9502
3      1995-08-19 00:00:00    Liverpool  ...          0   9503
4      1995-08-19 00:00:00     Man City  ...          0   9504
...                    ...          ...  ...        ...    ...
11194             26/12/24  Southampton  ...          0   9294
11195             26/12/24       Wolves  ...          0   9295
11196             26/12/24    Liverpool  ...          0   9296
11197             27/12/24     Brighton  ...          0   9297
11198             27/12/24      Arsenal  ...          0   9298

[11199 rows x 49 columns]

In [3]:
# Filtering to include ARS v MCI matches only
ars_v_mci = data[
    ((data['HomeTeam'] == 'Arsenal') & (data['AwayTeam'] == 'Man City')) | 
    ((data['AwayTeam'] == 'Arsenal') & (data['HomeTeam'] == 'Man City'))]

In [4]:
# Number of games played (including 24/25 season first leg)
ars_v_mci.shape[0]

49

In [5]:
# Seasons games was played
ars_v_mci['Season'].unique()

array(['9596', '0001', '0203', '0304', '0405', '0506', '0607', '0708',
       '0809', '0910', '1011', '1112', '1213', '1314', '1415', '1516',
       '1617', '1718', '1819', '1920', '2021', '2122', '2223', '2324',
       '2425'], dtype=object)

In [6]:
ars_v_mci['BTTS'] = ((ars_v_mci['FTHG'] > 0) & (ars_v_mci['FTAG'] > 0)).astype(str)

In [7]:
# h2h historical record (ARS as a home team)
# H - Arsenal win, A - City win, D - draw

ars_v_mci[ars_v_mci['HomeTeam'] == 'Arsenal']['FTR'].value_counts()

FTR
H    11
A     7
D     6
Name: count, dtype: int64

In [8]:
# h2h historical record (MCI as a home team)
# H - City win, A - Arsenal win, D - draw

ars_v_mci[ars_v_mci['HomeTeam'] == 'Man City']['FTR'].value_counts()

FTR
H    12
A     9
D     4
Name: count, dtype: int64

In [9]:
# overall h2h historical record

ars_v_mci.groupby('HomeTeam')['FTR'].value_counts()

HomeTeam  FTR
Arsenal   H      11
          A       7
          D       6
Man City  H      12
          A       9
          D       4
Name: count, dtype: int64

In [10]:
# Home, away team win and draw to games played overall ratio (ARS v MCI)

round(ars_v_mci['FTR'].value_counts()/ars_v_mci.shape[0],2)

FTR
H    0.47
A    0.33
D    0.20
Name: count, dtype: float64

In [11]:
# h2h all time PL record

team1 = "Arsenal"
team2 = "Man City"

team1_wins = 0
team2_wins = 0
draws = 0

for _, row in ars_v_mci.iterrows():
    if row['HomeTeam'] == team1 and row['FTR'] == 'H':
        team1_wins += 1
    elif row['AwayTeam'] == team1 and row['FTR'] == 'A':
        team1_wins += 1
    elif row['HomeTeam'] == team2 and row['FTR'] == 'H':
        team2_wins += 1
    elif row['AwayTeam'] == team2 and row['FTR'] == 'A':
        team2_wins += 1
    else:
        draws += 1

print(f"Head-to-Head Record: {team1} vs {team2}")
print(f"{team1}: {team1_wins} wins")
print(f"{team2}: {team2_wins} wins")
print(f"Draws: {draws}")
print(f"Total Matches: {len(ars_v_mci)}")

Head-to-Head Record: Arsenal vs Man City
Arsenal: 20 wins
Man City: 19 wins
Draws: 10
Total Matches: 49


In [12]:
ars_goals = 0
mci_goals = 0

for _, row in ars_v_mci.iterrows():
    if row['HomeTeam'] == team1:
        ars_goals += row['FTHG']
        mci_goals += row['FTAG']
    else:
        ars_goals += row['FTAG']
        mci_goals += row['FTHG']
    
print(f"Head-to-Head Goal Record: {team1} vs {team2}")
print(f"{team1}: {ars_goals} goals scored")
print(f"{team2}: {mci_goals} goals scored")     
print(f"{team1}: {round(ars_goals/ars_v_mci.shape[0],2)} goals scored per game on avg")
print(f"{team2}: {round(mci_goals/ars_v_mci.shape[0],2)} goals scored per game on avg")     

Head-to-Head Goal Record: Arsenal vs Man City
Arsenal: 69.0 goals scored
Man City: 72.0 goals scored
Arsenal: 1.41 goals scored per game on avg
Man City: 1.47 goals scored per game on avg


In [13]:
# Filtering for a games where table diff exactly 2 positions

ars_v_mci[(ars_v_mci['Pos_diff'] == 2) | (ars_v_mci['Pos_diff'] == -2)][['HomeTeam', 'AwayTeam', 'HomeTeam_curpos', 'AwayTeam_curpos', 'FTHG', 'FTAG', 'Season']]

HomeTeam  AwayTeam  HomeTeam_curpos  AwayTeam_curpos  FTHG  FTAG Season
5358  Man City   Arsenal                5                3   4.0   2.0   0910
5668   Arsenal  Man City                3                5   0.0   0.0   0910
6236  Man City   Arsenal                1                3   1.0   0.0   1112
6397   Arsenal  Man City                3                1   1.0   0.0   1112
6506  Man City   Arsenal                2                4   1.0   1.0   1213
6676   Arsenal  Man City                4                2   0.0   2.0   1213
7769   Arsenal  Man City                2                4   2.0   1.0   1516
7964  Man City   Arsenal                4                2   2.0   2.0   1516
8147  Man City   Arsenal                3                5   2.0   1.0   1617
8271   Arsenal  Man City                5                3   2.0   2.0   1617

For the third time ever Arsenal is stepping onto the pitch as a second - placed team while Man City is fourth.

In [14]:
ars_v_mci['Goals_overall'] = ars_v_mci[['FTHG', 'FTAG']].sum(axis=1)

In [15]:
# Game with highest number of goals scored overall (9) 

ars_v_mci.loc[ars_v_mci['Goals_overall'].idxmax()]

Date                         2013-12-14 00:00:00
HomeTeam                                Man City
AwayTeam                                 Arsenal
FTHG                                         6.0
Game_settled                                 1.0
FTAG                                         3.0
FTR                                            H
HTHG                                         2.0
HTAG                                         1.0
Season                                      1314
HomeTeam_curpts                               29
AwayTeam_curpts                               35
HomeTeam_curpos                                1
AwayTeam_curpos                                4
Pos_diff                                      -3
Pos_diff_abs                                   3
Pos_diff_bin                                  -1
Pos_diff_std                           -0.358096
Table_winner                                   1
HomeTeam_curpts_avg                         1.93
AwayTeam_curpts_avg 

In [16]:
# Creating result column 

ars_v_mci['Result'] = ars_v_mci['FTHG'].astype(int).astype(str) + ' - ' + ars_v_mci['FTAG'].astype(int).astype(str)

In [17]:
ars_v_mci.groupby('Result')['Result'].value_counts()

Result
0 - 0    3
0 - 1    3
0 - 2    3
0 - 3    3
0 - 4    1
1 - 0    7
1 - 1    3
1 - 2    2
1 - 3    3
1 - 5    1
2 - 0    1
2 - 1    4
2 - 2    4
3 - 0    2
3 - 1    4
4 - 1    1
4 - 2    1
5 - 0    2
6 - 3    1
Name: count, dtype: int64

As you can see the most common result is home team win 1-0

In [18]:
bins = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 20]
labels = ['under 0.5', 'under 1.5', 'under 2.5', 'under 3.5', 'under 4.5', 'under 5.5', 'under 6.5', 'rest']

In [19]:
ars_v_mci['Goals_numb_bin'] = pd.cut(ars_v_mci['Goals_overall'], bins = bins, labels = labels)

In [20]:
# Group by 'Goals_over', get value counts, and filter out counts of 0
filtered = ars_v_mci.groupby('Goals_numb_bin')['Goals_overall'].value_counts()

# Drop rows with 0 counts
filtered = filtered[filtered > 0]

# Optionally, reset index for cleaner presentation
filtered = filtered.reset_index(name='count')
filtered['Cumulative_count'] = filtered['count'].cumsum()
filtered['Ratio'] = round(filtered['Cumulative_count']/ars_v_mci.shape[0], 2)

print(filtered)

  Goals_numb_bin  Goals_overall  count  Cumulative_count  Ratio
0      under 0.5            0.0      3                 3   0.06
1      under 1.5            1.0     10                13   0.27
2      under 2.5            2.0      7                20   0.41
3      under 3.5            3.0     11                31   0.63
4      under 4.5            4.0     12                43   0.88
5      under 5.5            5.0      3                46   0.94
6      under 6.5            6.0      2                48   0.98
7           rest            9.0      1                49   1.00


In [21]:
s_2425 = data[data['Season'] == '2425']
s_2425 = s_2425.reset_index()

In [22]:
s_2425_ars_only = s_2425[
    ((s_2425['HomeTeam'] == 'Arsenal') | 
    (s_2425['AwayTeam'] == 'Arsenal')) & pd.notna(s_2425['FTR'])]

In [23]:
s_2425_mci_only = s_2425[
    ((s_2425['HomeTeam'] == 'Man City') | 
    (s_2425['AwayTeam'] == 'Man City')) & pd.notna(s_2425['FTR'])]

In [24]:
s_2425_ars_only['BTTS'] = ((s_2425_ars_only['FTHG'] > 0) 
                           & (s_2425_ars_only['FTAG'] > 0)).astype(str)
s_2425_mci_only['BTTS'] = ((s_2425_mci_only['FTHG'] > 0) 
                           & (s_2425_mci_only['FTAG'] > 0)).astype(str)

In [25]:
# Arsenal this season BTTS

s_2425_ars_only.groupby('BTTS')['BTTS'].value_counts()

BTTS
False    9
True     9
Name: count, dtype: int64

In [26]:
# Man City this season BTTS

s_2425_mci_only.groupby('BTTS')['BTTS'].value_counts()

BTTS
False     5
True     13
Name: count, dtype: int64

In [27]:
# BTTS all time ARS v MCI

ars_v_mci.groupby('BTTS')['BTTS'].value_counts()

BTTS
False    25
True     24
Name: count, dtype: int64

In [28]:
s_2425.loc[[169,178], ['HomeTeam', 'HomeTeam_curpts', 'HomeTeam_curpts_avg', 'HomeTeam_avg_scored', 'HomeTeam_ratio']]

HomeTeam  HomeTeam_curpts  ...  HomeTeam_avg_scored  HomeTeam_ratio
169  Man City               27  ...                 1.67            0.22
178   Arsenal               33  ...                 1.94            0.33

[2 rows x 5 columns]

In [29]:
city_l = data[((data['HomeTeam'] == 'Man City') & (data['FTAG'] >=5)) | ((data['AwayTeam'] == 'Man City') & (data['FTHG'] >=5))][['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'Season']]

city_l['Result'] = city_l['FTHG'].astype(int).astype(str) + ' - ' + city_l['FTAG'].astype(int).astype(str)

city_l[['Date', 'HomeTeam', 'AwayTeam', 'Result', 'Season']]

Date       HomeTeam   AwayTeam Result Season
103   1995-10-28 00:00:00      Liverpool   Man City  6 - 0   9596
1999  2000-10-28 00:00:00        Arsenal   Man City  5 - 0   0001
2934  2003-02-22 00:00:00       Man City    Arsenal  1 - 5   0203
2961  2003-03-22 00:00:00        Chelsea   Man City  5 - 0   0203
4658  2007-10-27 00:00:00        Chelsea   Man City  6 - 0   0708
4934  2008-11-05 00:00:00  Middlesbrough   Man City  8 - 1   0708
9524  2021-08-28 00:00:00       Man City  Leicester  2 - 5   2021

In [30]:
data[((data['HomeTeam'] == 'Arsenal') & (data['FTHG'] >= 5) & (data['AwayTeam_curpos'] <=4)) | ((data['AwayTeam'] == 'Arsenal') & (data['FTAG'] >= 5) & (data['HomeTeam_curpos'] <=4))]

Date   HomeTeam  ... Bet_winner  Index
3789  2005-11-05 00:00:00    Arsenal  ...          0   1889
6336  2012-02-26 00:00:00    Arsenal  ...          1   4436
7660  2015-09-26 00:00:00  Leicester  ...          0   5760

[3 rows x 49 columns]

In [32]:
data['HomeTeam2ndHalfGoals'] = data['FTHG'] - data['HTHG']
data['AwayTeam2ndHalfGoals'] = data['FTAG'] - data['HTAG']
data

Date  ... AwayTeam2ndHalfGoals
0      1995-08-19 00:00:00  ...                  1.0
1      1995-08-19 00:00:00  ...                  0.0
2      1995-08-19 00:00:00  ...                  0.0
3      1995-08-19 00:00:00  ...                  0.0
4      1995-08-19 00:00:00  ...                  0.0
...                    ...  ...                  ...
11194             26/12/24  ...                  1.0
11195             26/12/24  ...                  0.0
11196             26/12/24  ...                  0.0
11197             27/12/24  ...                  0.0
11198             27/12/24  ...                  0.0

[11199 rows x 51 columns]

In [34]:
data[((data['HomeTeam'] == 'Arsenal') & (data['HomeTeam2ndHalfGoals'] >= 4)) | ((data['AwayTeam'] == 'Arsenal') & (data['AwayTeam2ndHalfGoals'] >= 4))][['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HomeTeam2ndHalfGoals', 'AwayTeam2ndHalfGoals']]

Date  ... AwayTeam2ndHalfGoals
439    1996-09-16 00:00:00  ...                  0.0
1471   1999-04-19 00:00:00  ...                  1.0
1919   2000-08-26 00:00:00  ...                  1.0
1999   2000-10-28 00:00:00  ...                  0.0
2089   2000-12-26 00:00:00  ...                  1.0
3438   2004-08-22 00:00:00  ...                  2.0
3546   2004-11-13 00:00:00  ...                  4.0
3789   2005-11-05 00:00:00  ...                  0.0
4751   2007-12-29 00:00:00  ...                  4.0
4919   2008-04-28 00:00:00  ...                  4.0
5256   2009-11-04 00:00:00  ...                  4.0
6169   2011-10-29 00:00:00  ...                  4.0
6310   2012-04-02 00:00:00  ...                  0.0
6648   2012-12-29 00:00:00  ...                  2.0
6689   2013-01-23 00:00:00  ...                  0.0
7448   2015-01-02 00:00:00  ...                  0.0
8116   2016-03-12 00:00:00  ...                  4.0
8448   2017-10-22 00:00:00  ...                  4.0
8494   2017-11-29 00:00:00  ...                  0.0
8700   2018-04-22 00:00:00  ...                  1.0
8817   2018-07-10 00:00:00  ...                  4.0
9377   2020-02-16 00:00:00  ...                  0.0
10384  2022-10-30 00:00:00  ...                  0.0
10732  2023-10-28 00:00:00  ...                  0.0
10970  2024-04-23 00:00:00  ...                  0.0

[25 rows x 9 columns]